# Datasets

We use Google Drive to share the data used for training the model and the model already trained. To add a shortcut to the location of the data in your Google Drive, follow these steps:    

* Go to https://drive.google.com/drive/folders/1EPfe4G6Y32c6eTrM6gnTjxr7KUlrGwAt?usp=sharing
* Click in "MAPBIOMAS-PUBLIC" &#8594; "Add Shortcut to Drive" &#8594; "My Drive" &#8594; "ADD SHORTCUT"

# Mount Google Drive

After adding the shortcut to the data in your Google Drive, the next step is to mount a Google Drive volume on Google Colab.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [ ]:
!du -h -d 3 /content/drive/My\ Drive/MAPBIOMAS-PUBLIC/

Information about directories and files:


* collection_5/center_pivot_irrigation/train &#8594; contains images ending in "_mosaic.tif", with 3 bands, and images ending in "_labels.tif", with 1 band. 
For each image ending in "_mosaic.tif", which we use as an entry for the model, there is an image with the same prefix, but ending in "_labels.tif" containing the ground truth used to adjust the model;

* collection_5/center_pivot_irrigation/test &#8594; follows the same structure as before, but with images that were used to test the model after the training process;

* collection_5/center_pivot_irrigation/predict &#8594; example image that we can use, after training the model or using our trained model, to map the central pivot irrigation systems;

* collection_5/center_pivot_irrigation/logs &#8594; trained model and log files generated in the training process.

# Check GPU

We recommend that the entire model and classification training process be done using some of the GPUs available from Google Colab.

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

# Install Requirements

In [ ]:
#Step 1
!apt-get update
#Step 2
!apt-get install libgdal-dev -y
#Step 3
!apt-get install python-gdal -y

# Settings

## Center Pivot Irrigation Systems

In [ ]:
CHIP_SIZE = 256
CHANNELS = 3
LABELS = [0, 1]
SPATIAL_SCALE = 30
PROJECTION = 3857

# Build Datasets
GRIDS = 1
ROTATE = True
FLIP = False

TRAIN_VALIDATION_DIR = "/content/drive/My Drive/MAPBIOMAS-PUBLIC/collection_5/center_pivot_irrigation/train"
TEST_DIR = "/content/drive/My Drive/MAPBIOMAS-PUBLIC/collection_5/center_pivot_irrigation/test"
TRAIN_PATH = "/content/train.h5"
VALIDATION_PATH = "/content/validation.h5"
TEST_PATH = "/content/test.h5"

# Train model
TRAIN_BATCH_SIZE = 20
TRAIN_EPOCHS = 50

# Predict images
# PREDICT_INPUT_DIR = "/content/drive/My Drive/MAPBIOMAS-PUBLIC/collection_5/center_pivot_irrigation/predict"
PREDICT_INPUT_DIR = "/content/drive/My Drive/MAPBIOMAS-PRIVATE"
PREDICT_OUPUT_DIR = "/content/drive/My Drive/MAPBIOMAS-PRIVATE/RESULTS"
PREDICT_CHIP_SIZE = 1024
PREDICT_GRIDS = 3
PREDICT_BATCH_SIZE = 1

If you want to train a new model, you must run the following code snippet.

In [ ]:
MODEL_DIR = "/content/drive/My Drive/MAPBIOMAS-PRIVATE/logs"
TRAIN_NEW_MODEL = True

If you want to use the model trained by us, just to perform the classification process, you must execute the code below. 

As the directory where the trained model will be published in read-only mode, if you execute this code snippet and try to retrain the model, you will receive an error indicating that Google Colab is unable to save the new trained model.

In [ ]:
MODEL_DIR = "/content/drive/My Drive/MAPBIOMAS-PUBLIC/collection_5/center_pivot_irrigation/logs"
TRAIN_NEW_MODEL = False

# Image Utils

In [ ]:
import gc
import os

import h5py
import numpy as np
from osgeo import gdal, osr
from skimage.transform import resize, rotate
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from tqdm import tqdm


def load_file(path, norm=False):
    original_source = gdal.Open(path)
    new_source = reproject_dataset(original_source,
                                   pixel_spacing=SPATIAL_SCALE,
                                   epsg_to=PROJECTION)

    if not new_source is None:
        bands = []
        for index in range(1, new_source.RasterCount + 1):
            band = new_source.GetRasterBand(index).ReadAsArray()
            if norm:
                band = normalize(band)
            bands.append(band)

        image = np.dstack(bands)

        return original_source, new_source, image
    else:
        return original_source, new_source, None

def normalize(image):
    image_max = float(np.max(image))
    image_min = float(np.min(image))
    normalized = (image - image_min) / (image_max - image_min)
    return normalized


def get_rotate(image):
    images = []
    for rot in [90, 180, 270]:
        image_rotate = rotate(image, rot, preserve_range=True)
        images.append(image_rotate)
    return images

def get_flip(image):
    horizontal_flip = image[:, ::-1]
    vertical_flip = image[::-1, :]
    return [horizontal_flip, vertical_flip]

def make_dataset(filename, width, height, channels):
    dataset = h5py.File(filename, 'w')
    x_data = dataset.create_dataset("x", (0, width, height, channels), 'f',
                                    maxshape=(None, width, height, channels),
                                    chunks=True)
    y_data = dataset.create_dataset("y", (0, width, height, 1), 'f',
                                    maxshape=(None, width, height, 1),
                                    chunks=True)
    return dataset, x_data, y_data

def save_dataset(X, y, output_path, chip_size, channels):
    if os.path.isfile(output_path):
        dataset, x_data, y_data = load_dataset(output_path)
    else:
        dataset, x_data, y_data = make_dataset(output_path, chip_size,
                                               chip_size, channels)

    length = len(X)

    x_data_size = x_data.len()
    y_data_size = y_data.len()

    x_data.resize((x_data_size + length, chip_size, chip_size, channels))
    y_data.resize((y_data_size + length, chip_size, chip_size, 1))

    print("Saving dataset...")

    for index in tqdm(iterable=range(length), miniters=10, unit=" samples"):
        x_data[x_data_size + index] = X[index]
        y_data[y_data_size + index] = y[index]

    dataset.close()

def load_dataset(dataset, read_only=False):
    if read_only:
        dataset = h5py.File(dataset, 'r')
    else:
        dataset = h5py.File(dataset, 'r+')
    
    x_data = dataset["x"]
    y_data = dataset["y"]

    return dataset, x_data, y_data

def chip_is_empty(chip):
    labels_unique = np.unique(chip)
    return 0 in labels_unique and len(labels_unique) == 1


def generate_dataset(image_path, labels_path, chip_size, channels, 
                     grids=1, allow_empty_chip=False, rotate=False, flip=False):
    _, _, image_data = load_file(image_path, norm=True)
    _, _, image_labels = load_file(labels_path)

    image_labels = resize(image_labels,
                          (image_data.shape[0], image_data.shape[1]),
                          preserve_range=True, anti_aliasing=True).astype(np.int8)

    image = np.dstack([image_data, image_labels])

    X_set = []
    y_set = []
    for step in get_grids(grids, chip_size):
        for (x, y, window, dimension) in sliding_window(image,
                                                        step["steps"],
                                                        step["chip_size"],
                                                        (chip_size,
                                                         chip_size)):

            train = np.array(window[:, :, : channels], dtype=np.float16)
            labels = np.array(window[:, :, -1:], dtype=np.int8)

            if chip_is_empty(labels) and not allow_empty_chip:
                continue

            raw_image = np.dstack([train, labels])
            images_daugmentation = [raw_image]

            if rotate:
                images_rotate = get_rotate(raw_image)
                images_daugmentation.extend(images_rotate)

            if flip:
                images_flip = []
                for im in images_daugmentation:
                    images_flip.extend(get_flip(im))
                images_daugmentation.extend(images_flip)

            X_group = []
            Y_group = []

            for i in images_daugmentation:
                new_train = np.array(i[:, :, :channels], dtype=np.float16)
                new_labels = np.array(i[:, :, -1:], dtype=np.int8)

                np.clip(new_labels, 0, None, out=new_labels)
                
                X_group.append(new_train)
                Y_group.append(new_labels)

            X_set.append(X_group)
            y_set.append(Y_group)

        X_set = np.array(X_set)
        y_set = np.array(y_set)

        yield X_set, y_set


def generate_train_validation_dataset(image_path, labels_path, 
                                      train_path, validation_path, 
                                      chip_size, 
                                      channels=1, 
                                      grids=1, 
                                      allow_empty_chip=False, 
                                      rotate=False, flip=False):
    
    for X_set, y_set in generate_dataset(image_path, labels_path, 
                                    chip_size=chip_size,
                                    channels=channels, 
                                    grids=grids, 
                                    allow_empty_chip=allow_empty_chip, 
                                    rotate=rotate, 
                                    flip=flip):
        
        if len(X_set) >= 5:
            X_train, X_val, y_train, y_val = train_test_split(X_set, y_set,
                                                        test_size=0.25,
                                                        random_state=1)

            X_train =  np.array([item for sublist in X_train for item in sublist])
            y_train =  np.array([item for sublist in y_train for item in sublist])

            X_val =  np.array([item for sublist in X_val for item in sublist])
            y_val =  np.array([item for sublist in y_val for item in sublist])

            save_dataset(X_train, y_train, train_path, chip_size, channels)
            save_dataset(X_val, y_val, validation_path, chip_size, channels)

def generate_test_dataset(image_path, labels_path, test_path,
                                      chip_size=None, 
                                      channels=None, grids=1, 
                                      allow_empty_chip=False, 
                                      rotate=False, flip=False, *args):
    
    for X_set, y_set in generate_dataset(image_path, labels_path, 
                                    chip_size=chip_size,
                                    channels=channels, 
                                    grids=grids, 
                                    allow_empty_chip=allow_empty_chip, 
                                    rotate=rotate, 
                                    flip=flip):

        X_test =  np.array([item for sublist in X_set for item in sublist])
        y_test =  np.array([item for sublist in y_set for item in sublist])

        save_dataset(X_test, y_test, test_path, chip_size, channels)

def sliding_window(image, step, chip_size, chip_resize):
    # slide a chip across the image
    step_cols = int(step[0])
    step_rows = int(step[1])

    cols = image.shape[1]
    rows = image.shape[0]

    chip_size_cols = chip_size[0]
    chip_size_rows = chip_size[1]

    chip_resize_cols = chip_resize[0]
    chip_resize_rows = chip_resize[1]

    for y in range(0, rows, step_rows):
        for x in range(0, cols, step_cols):

            origin_x = x
            origin_y = y

            if (origin_y + chip_size_rows) > rows:
                origin_y = rows - chip_size_rows

            if (origin_x + chip_size_cols) > cols:
                origin_x = cols - chip_size_cols

            chip = image[origin_y:origin_y + chip_size_rows,
                   origin_x: origin_x + chip_size_cols]

            original_shape = chip.shape

            if chip.shape != (chip_resize_cols, chip_resize_rows):
                chip = resize(chip,
                              (chip_resize_cols, chip_resize_rows),
                              preserve_range=True,
                              anti_aliasing=True).astype(np.float16)

            yield (origin_x, origin_y, chip, original_shape)


def get_window(matrix, x, y, width, height):
    return matrix[y:y + height, x:x + width]


def set_window(matrix, x, y, new_matrix):
    for i_index, i in enumerate(range(y, y + new_matrix.shape[0])):
        for j_index, j in enumerate(range(x, x + new_matrix.shape[1])):
            matrix[i][j] = new_matrix[i_index][j_index]


def transform_labels(labels_array, labels):
    lb = preprocessing.LabelBinarizer()
    lb.fit(labels)
    new_labels_array = []
    for ix, l in enumerate(labels_array):
        flat_labels = l.reshape((l.shape[0] * l.shape[1],))
        transformed_flat_labels = lb.transform(flat_labels)
        new_labels_array.append(transformed_flat_labels.reshape(
            (l.shape[0], l.shape[1], len(labels))))

    new_labels_array = np.array(new_labels_array)
    return new_labels_array


def get_grids(grids, chip_size):
    grids_dict = {
        1: [
            {"steps": (chip_size, chip_size),
             "chip_size": (chip_size, chip_size)}
        ],
        2: [
            {"steps": (int(chip_size * 0.5), int(chip_size * 0.5)),
             "chip_size": (chip_size, chip_size)},
        ],
        3: [
            {"steps": (int(chip_size * 0.9), int(chip_size * 0.9)),
             "chip_size": (chip_size, chip_size)},
        ]
    }

    return grids_dict[grids]


def reproject_dataset(g, pixel_spacing=30., epsg_to=3857):
    osng = osr.SpatialReference()
    osng.ImportFromEPSG(epsg_to)

    wkt = g.GetProjection()
    wgs84 = osr.SpatialReference()
    wgs84.ImportFromWkt(wkt)

    tx = osr.CoordinateTransformation(wgs84, osng)
    # Up to here, all  the projection have been defined, as well as a
    # transformation from the from to the  to :)

    # Get the Geotransform vector
    geo_t = g.GetGeoTransform()
    x_size = g.RasterXSize  # Raster xsize
    y_size = g.RasterYSize  # Raster ysize
    # Work out the boundaries of the new dataset in the target projection
    (ulx, uly, ulz) = tx.TransformPoint(geo_t[0], geo_t[3])
    (lrx, lry, lrz) = tx.TransformPoint(geo_t[0] + geo_t[1] * x_size, \
                                        geo_t[3] + geo_t[5] * y_size)

    # Now, we create an in-memory raster
    mem_drv = gdal.GetDriverByName('MEM')
    # The size of the raster is given the new projection and pixel spacing
    # Using the values we calculated above. Also, setting it to store one band
    # and to use Float32 data type.
    dest = mem_drv.Create('', int((lrx - ulx) / pixel_spacing), \
                          int((uly - lry) / pixel_spacing), g.RasterCount,
                          g.GetRasterBand(1).DataType)
    # Calculate the new geotransform
    new_geo = (ulx, pixel_spacing, geo_t[2], \
               uly, geo_t[4], -pixel_spacing)
    # Set the geotransform
    dest.SetGeoTransform(new_geo)
    dest.SetProjection(osng.ExportToWkt())
    # Perform the projection/resampling
    res = gdal.ReprojectImage(g, dest, \
                              wgs84.ExportToWkt(), osng.ExportToWkt(), \
                              gdal.GRA_NearestNeighbour)
    return dest



def save_results(original_dataset, reprojected_dataset, image, output_path):
    mem_dataset = reprojected_dataset \
        .GetDriver() \
        .Create(output_path, image.shape[1], image.shape[0], 1, gdal.GDT_Int16)

    mem_dataset.SetGeoTransform(reprojected_dataset.GetGeoTransform())
    mem_dataset.SetProjection(reprojected_dataset.GetProjection())
    mem_dataset.GetRasterBand(1) \
        .WriteArray(image.reshape((image.shape[0], image.shape[1])), 0, 0)
    mem_dataset.FlushCache()

    original_epsg = int(osr \
                        .SpatialReference(wkt=original_dataset.GetProjection()) \
                        .GetAttrValue('AUTHORITY', 1))

    output_dataset = reproject_dataset(mem_dataset,
                                       SPATIAL_SCALE,
                                       original_epsg)

    original_dataset.GetDriver().CreateCopy(output_path,
                                            output_dataset,
                                            options=['COMPRESS=LZW',
                                                     'TFW=YES'])

# Model

## Evaluation Metric

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K

def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred = K.cast(K.greater(y_pred, t), dtype='float32')
        inter = K.sum(K.sum(K.squeeze(y_true * y_pred, axis=3), axis=2), axis=1)
        union = K.sum(K.sum(K.squeeze(y_true + y_pred, axis=3), axis=2), axis=1) - inter
        acc = K.mean((inter + K.epsilon()) / (union + K.epsilon()))
        prec.append(acc)
    return  K.mean(tf.convert_to_tensor(prec))

## U-Net

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, Dropout, MaxPooling2D, \
    Conv2DTranspose, concatenate, BatchNormalization, Activation


def conv(n_filters, kernel_size, activation='elu', inputs=None):
    net = Conv2D(n_filters, kernel_size, activation=None, \
                 kernel_initializer='he_normal', padding='same') (inputs) 
    net = BatchNormalization()(net)
    net = Activation(activation)(net)
    return net

def transpose(n_filters, kernel_size, activation='elu', inputs=None):
    net = Conv2DTranspose(n_filters, kernel_size, activation=None, 
                          strides=(2, 2), padding='same', 
                          kernel_initializer='he_normal') (inputs)
    net = BatchNormalization()(net)
    net = Activation(activation)(net)
    return net

def model_fn(input_shape, n_filters=64, labels=[]):
    print(input_shape, labels)
    inputs = keras.Input(input_shape)

    c1 = conv(n_filters * 1, (3, 3), inputs=inputs)
    c1 = conv(n_filters * 1, (3, 3), inputs=c1)
    p1 = MaxPooling2D((2, 2)) (c1)
    p1 = Dropout(0.25) (p1)

    c2 = conv(n_filters * 2, (3, 3), inputs=p1)
    c2 = conv(n_filters * 2, (3, 3), inputs=c2)
    p2 = MaxPooling2D((2, 2)) (c2)
    p2 = Dropout(0.25) (p2)

    c3 = conv(n_filters * 4, (3, 3), inputs=p2)
    c3 = conv(n_filters * 4, (3, 3), inputs=c3)
    p3 = MaxPooling2D((2, 2)) (c3)
    p3 = Dropout(0.5) (p3)

    c4 = conv(n_filters * 8, (3, 3), inputs=p3)
    c4 = conv(n_filters * 8, (3, 3), inputs=c4)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)
    p4 = Dropout(0.5) (p4)

    c5 = conv(n_filters * 16, (3, 3), inputs=p4)
    c5 = conv(n_filters * 16, (3, 3), inputs=c5)
    c5 = Dropout(0.5) (c5)

    u6 = transpose(n_filters * 8, (2, 2), inputs=c5)
    u6 = concatenate([u6, c4])
    c6 = conv(n_filters * 8, (2, 2), inputs=u6)
    c6 = conv(n_filters * 8, (2, 2), inputs=c6)
    c6 = Dropout(0.5) (c6)
    
    u7 = transpose(n_filters * 4, (2, 2), inputs=c6)
    u7 = concatenate([u7, c3])
    c7 = conv(n_filters * 4, (2, 2), inputs=u7)
    c7 = conv(n_filters * 4, (2, 2), inputs=c7)
    c7 = Dropout(0.5) (c7)

    u8 = transpose(n_filters * 2, (2, 2), inputs=c7)
    u8 = concatenate([u8, c2])
    c8 = conv(n_filters * 2, (2, 2), inputs=u8)
    c8 = conv(n_filters * 2, (2, 2), inputs=c8)
    c8 = Dropout(0.25) (c8)
    
    u9 = transpose(n_filters * 1, (2, 2), inputs=c8)
    u9 = concatenate([u9, c1], axis=3)
    c9 = conv(n_filters * 1, (2, 2), inputs=u9)
    c9 = conv(n_filters * 1, (2, 2), inputs=c9)
    c9 = Dropout(0.25) (c9)
    
    if len(labels) > 2:
        outputs = tf.keras.layers.Conv2D(len(labels), 1, 1, activation='softmax')(c9)
        loss_function = 'categorical_crossentropy'
        metrics = ['categorical_accuracy']
    else:
        outputs = conv(1, (1, 1), activation='sigmoid', inputs=c9)
        loss_function = 'binary_crossentropy'
        metrics = [mean_iou]

    model = keras.Model(inputs=inputs, outputs=outputs)
    optimizer = tf.keras.optimizers.Nadam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)

    return model

# Classifier

In [ ]:
import numpy as np
import tensorflow as tf
from osgeo import gdal
from tqdm import tqdm


class Classifier(object):
    def __init__(self, chip_size, channels, model_dir, labels):
        self.__chip_size = chip_size
        self.__channels = channels
        self.__labels = labels
        self.__model = model_fn((chip_size, chip_size, channels), labels=labels)
        self.__checkpoints = []

        if not os.path.exists(model_dir):
            os.makedirs(model_dir)

        self.load_model(model_dir)
        self.load_callbacks(model_dir)

    def load_model(self, model_dir):
        latest = tf.train.latest_checkpoint(model_dir)

        if latest:
            print("Loading model....")
            self.__model.load_weights(latest)

        print("Model loaded!")
        self.__model.summary()

    def load_callbacks(self, model_dir):
        checkpoint_path = "{dir}/model.ckpt".format(dir=model_dir)

        cp_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_path,
            save_weights_only=True,
            save_best_only=True)

        tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=model_dir)

        self.__callbacks = [cp_callback, tensorboard_callback]


    def train(self, input_train, input_validation, epochs, batch_size):
        train_dataset, train_data, train_labels = load_dataset(input_train,
                                                            read_only=True)
        validation_dataset, validation_data, validation_labels = load_dataset(
            input_validation, read_only=True)

        train_images = np.asarray(train_data, dtype=np.float16)
        train_labels = np.asarray(train_labels, dtype=np.int8)

        validation_images = np.asarray(validation_data, dtype=np.float16)
        validation_labels = np.asarray(validation_labels, dtype=np.int8)

        if len(self.__labels) > 2:
            train_labels = transform_labels(train_labels, self.__labels)
            validation_labels = transform_labels(validation_labels,
                                                 self.__labels)

        print(train_images.shape)
        print(train_labels.shape)

        print("\nTrain: {train_size}\n Validation: {validation_size}".format(
            train_size=train_images.shape[0], 
            validation_size=validation_images.shape[0]))

        self.__model.fit(x=train_images,
                         y=train_labels,
                         validation_data=(validation_images, validation_labels),
                         epochs=epochs,
                         batch_size=batch_size,
                         verbose=1,
                         callbacks=self.__callbacks,
                         shuffle=True)
        
        train_dataset.close()
        validation_dataset.close()

    def evaluate(self, input_test, batch_size):
        test_file, test_data, test_labels = load_dataset(input_test,
                                                         read_only=True)

        test_data = np.asarray(test_data, dtype=np.float16)
        test_labels = np.asarray(test_labels, dtype=np.int8)

        test_results = self.__model.evaluate(test_data,
                                             test_labels,
                                             batch_size=batch_size)

        print('test loss, test acc:', test_results)

    def predict(self, input_path, output_path, grids, batch_size):
        original_dataset, input_dataset, image = load_file(input_path,
                                                           norm=True)

        image = image[:, :, : self.__channels]

        predicted_image = np.zeros((image.shape[0], image.shape[1]),
                                   dtype=np.int8)

        grids = get_grids(grids, self.__chip_size)

        for step in grids:
            batch = []
            windows = sliding_window(image, step["steps"], step["chip_size"],
                                     (self.__chip_size, self.__chip_size))

            for (x, y, chip, original_dimensions) in tqdm(iterable=windows,
                                                          miniters=10,
                                                          unit=" windows"):

                normalized_chip = normalize(chip)

                batch.append({
                    "chip": normalized_chip,
                    "x": x,
                    "y": y,
                    "dimensions": original_dimensions
                })

                if len(batch) >= batch_size:
                    chips = []
                    positions = []
                    dimensions = []

                    for b in batch:
                        chips.append(b.get("chip"))
                        positions.append((b.get("x"), b.get("y")))
                        dimensions.append(b.get("dimensions"))

                    chips = np.array(chips, dtype=np.float16)

                    pred = self.__model.predict(chips, batch_size=batch_size)

                    for chip, position, dimension, predict in zip(chips,
                                                                  positions,
                                                                  dimensions,
                                                                  pred):

                        if len(self.__labels) > 2:
                            predict = np.array(tf.math.argmax(predict, axis=2))
                        else:
                            predict[predict > 0.5] = 1
                            predict[predict <= 0.5] = 0

                        predict = resize(predict, (dimension[0], dimension[1]),
                                         preserve_range=True,
                                         anti_aliasing=True).astype(np.int8)

                        predict = predict.reshape(
                            (predict.shape[0], predict.shape[1]))

                        predicted = get_window(predicted_image,
                                               position[0],
                                               position[1],
                                               predict.shape[1],
                                               predict.shape[0])

                        if predict.shape != predicted.shape:
                            raise Exception("predict.shape != predicted.shape")

                        if len(self.__labels) > 2:
                            set_window(predicted_image, position[0],
                                       position[1], predict)
                        else:
                            set_window(predicted_image, position[0],
                                       position[1], np.add(predict, predicted))

                    batch = []

            if len(self.__labels) == 2:
                predicted_image[predicted_image >= 1] = 1
                
            print("Saving results...")
            save_results(original_dataset, input_dataset, predicted_image,
                      output_path)
            print("Finished!")

# Train Model

## Build Train and Validation datasets

In [ ]:
if TRAIN_NEW_MODEL:
    import glob

    images = [f for f in glob.glob(TRAIN_VALIDATION_DIR + "/*mosaic.tif", recursive=True)]

    if len(images) == 0:
        print("No samples found.")

    for image_path in images:
        labels_path = image_path.replace("mosaic", "labels")
        print(image_path)
        print(labels_path)
        generate_train_validation_dataset(
            image_path=image_path,
            labels_path=labels_path,
            train_path=TRAIN_PATH,
            validation_path=VALIDATION_PATH,
            chip_size=CHIP_SIZE,
            channels=CHANNELS,
            grids=GRIDS,
            rotate=ROTATE
        )
else:
    print("Please, go to the settings and execute the code snippet indicated for training a new model.")

## Plot Images

In [ ]:
import matplotlib.pyplot as plt
import cv2
import random


def plot_figures(figures, nrows = 1, ncols=1):
    """Plot a dictionary of figures.

    Parameters
    ----------
    figures : <title, figure> dictionary
    ncols : number of columns of subplots wanted in the display
    nrows : number of rows of subplots wanted in the figure
    """

    fig, axeslist = plt.subplots(ncols=ncols, nrows=nrows, figsize=(19, 10))
    for ind,title in zip(range(len(figures)), figures):
        image = figures[title]
        if image.shape[2] >= 3:
            image = image[:, : , :3]
            axeslist.ravel()[ind].imshow(image, vmin=0.1, vmax=0.4)
        else:
            image = image.reshape((image.shape[0], image.shape[1]))
            axeslist.ravel()[ind].imshow(image)
        axeslist.ravel()[ind].set_title(title)
        axeslist.ravel()[ind].set_axis_off()
    plt.tight_layout() # optional

figures = {}

dataset, x, y = load_dataset(TRAIN_PATH, read_only=True)

ids = []

samples = 4

for i in range(0, samples):
    id = random.randint(0,x.shape[0])
    figures['img_x'+ str(i)] = x[id]
    figures['img_y'+ str(i)] = y[id]

plot_figures(figures, 2, samples)
dataset.close()

## Run Train

In [ ]:
if TRAIN_NEW_MODEL:
    classifier = Classifier(chip_size=CHIP_SIZE,
                            channels=CHANNELS,
                            model_dir=MODEL_DIR,
                            labels=LABELS)

    classifier.train(
        input_train=TRAIN_PATH,
        input_validation=VALIDATION_PATH,
        epochs=TRAIN_EPOCHS,
        batch_size=TRAIN_BATCH_SIZE
    )
else:
    print("Please, go to the settings and execute the code snippet indicated for training a new model.")

# Evaluate Model

## Build Test dataset

In [ ]:
import glob

images = [f for f in glob.glob(TEST_DIR + "/*mosaic.tif", recursive=True)]

if len(images) == 0:
    print("No test found.")

for image_path in images:
    labels_path = image_path.replace("mosaic", "labels")
    print(image_path)
    print(labels_path)
    generate_test_dataset(
        image_path=image_path,
        labels_path=labels_path,
        test_path=TEST_PATH,
        chip_size=CHIP_SIZE,
        channels=CHANNELS,
        grids=1,
        allow_empty_chip=True
    )

## Run Evaluation

In [ ]:
classifier = Classifier(chip_size=CHIP_SIZE,
                        channels=CHANNELS,
                        model_dir=MODEL_DIR,
                        labels=LABELS)

classifier.evaluate(
    input_test=TEST_PATH,
    batch_size=BATCH_SIZE
)

# Predict Images

In [ ]:
from os import listdir
from os.path import isfile, join, sep, exists

classifier = Classifier(chip_size=PREDICT_CHIP_SIZE,
                        channels=CHANNELS,
                        model_dir=MODEL_DIR,
                        labels=LABELS)

files = [f for f in listdir(PREDICT_INPUT_DIR) if
         isfile(join(PREDICT_INPUT_DIR, f))]

if len(files) == 0:
    print("No file found.")

for f in files:
    print("File:", f)
    input_file = "{directory}/{filepath}".format(directory=PREDICT_INPUT_DIR, filepath=f)

    output_file = "{directory}/{filepath}".format(directory=PREDICT_OUPUT_DIR, filepath=f)

    if not os.path.exists(PREDICT_OUPUT_DIR):
            os.makedirs(PREDICT_OUPUT_DIR)

    if exists(output_file):
        print("File {} exists. Skipping...".format(output_file))
        continue

    print("Predict: ", input_file, "  >>  ", output_file)

    classifier.predict(input_path=input_file, output_path=output_file,
                       grids=PREDICT_GRIDS,
                       batch_size=PREDICT_BATCH_SIZE)

# Do it yourself

What do you think about mapping central pivot irrigation systems anywhere in the world?

First, you will need to build an image that will be used as an input to our mapping model. To build this image, you can use this [script](https://code.earthengine.google.com/cac251ac4de93c1c09fcbcb8adf00d7a) that must be run on the platform [Google Earth Engine](https://earthengine.google.com/).

Following the instructions in the script, it will export an image into the MAPBIOMAS-PRIVATE directory on your Google Drive.

With the image saved, you need to change the variable **PREDICT_INPUT_DIR** in the section **Settings** for the value: 

PREDICT_INPUT_DIR = "/content/drive/My Drive/MAPBIOMAS-PRIVATE"

After changing the variable value, just run the sections again
 **Settings** and **Predict Images** and get the mapping result in your Google Drive.

